In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob

In [14]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0], errors='coerce')
#     attendance = str(df.iloc[0,0]).split(' ',1)[1]
    return job, edit_date

def job_code(completed_path):
    files = []
    for item in os.listdir(completed_path):
        job_code = item.split(' ')[1]
        files.append(job_code)
    files= pd.unique(files)
    return files
    
    
def lat_old(files):
    file_list = []
    for file in files:
        job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
        latest_file = max(job_files, key=os.path.getctime)
        oldest_file = min(job_files, key=os.path.getctime)
        file_list.append((latest_file, oldest_file))
    file_list = pd.unique(file_list) 
    return file_list

def files_to_dfs(file_list):
    df_list = []
    for item in file_list:
        for file in item:
            num_sheet = len(pd.ExcelFile(file).sheet_names)
            for sheet in range(0,num_sheet):
                xl = pd.read_excel(file ,sheet_name=sheet)
                df = pd.DataFrame(xl)
                job, date = job_date(df)
                df.columns = df.iloc[2]
                df = df.drop(df.index[:3])
                df = df.iloc[:, :27]
                df['updated'] = date
                df['job'] = job
                df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
                df = df.dropna(subset=['Qty'])
                df = df.reset_index(drop=True)
                df_list.append(df)
    return df_list

In [3]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
c_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
on_going_path = ''

In [15]:
job_codes = job_code(completed_path)
lat_old_tuple = lat_old(job_codes)
df_list = files_to_dfs(lat_old_tuple)

In [4]:
# files = []
# for item in os.listdir(completed_path):
#     job_code = item.split(' ')[1]
#     files.append(job_code)
# files= pd.unique(files)
    
#     full_path = os.path.join(completed_path, item)

In [5]:
# latest_file_list = []
# for file in files:
#     job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
#     latest_file = max(job_files, key=os.path.getctime)
#     latest_file_list.append(latest_file)
# latest_file_list = pd.unique(latest_file_list)   

In [6]:
# df_list = []
# for item in latest_file_list:
#     num_sheet = len(pd.ExcelFile(item).sheet_names)
#     for sheet in range(0,num_sheet):
#         xl = pd.read_excel(item ,sheet_name=sheet)
#         df = pd.DataFrame(xl)
#         job, date = job_date(df)
#         df.columns = df.iloc[2]
#         df = df.drop(df.index[:3])
#         df = df.iloc[:, :27]
#         df['updated'] = date
#         df['job'] = job
#         df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
#         df = df.dropna(subset=['Qty'])
#         df = df.reset_index(drop=True)
#         df_list.append(df)

In [16]:
full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
full_df['Ph Int'] = full_df['Phase Code'].astype(str).str[0]
full_df = full_df[~full_df['updated'].isnull()]
full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
full_df['month'] = pd.DatetimeIndex(full_df['updated']).month
# full_df['Est unit $'] = full_df['Total Budget'] / full_df['Qty']
# full_df['Act unit $'] = full_df['Actual Cost$'] / full_df['HJ Qty Comp']
full_df = full_df.sort_values(by='updated', ascending=False)

In [19]:
full_df = full_df[~full_df['updated'].isnull()]
first = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='first')
last = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='last')


# no_dup['Qty'] = no_dup['Qty'].astype(float)
# no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
# no_dup['Total Budget'] = no_dup['Total Budget'].astype(float)

In [22]:
len(pd.unique(full_df['updated']))

62

In [28]:
len(full_df['updated'])
first

2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,updated,job,Ph Int,year,month
7737,7170,Electrical Building Foundation,NaN,1,0.0,EA,0,NaN,2192.41,NaN,...,0,NaN,1742.13,NaN,1742.13,2023-11-28,10378 Brunner Farm-CGRS,7,2023,11
7728,1060,Survey/GPS,NaN,0,0.0,HRS,NaN,NaN,0,NaN,...,0,NaN,0,NaN,0,2023-11-28,10378 Brunner Farm-CGRS,1,2023,11
7719,1000,Incidentals,NaN,0,0.0,LS,NaN,NaN,0,NaN,...,0,NaN,0,NaN,0,2023-11-28,10378 Brunner Farm-CGRS,1,2023,11
7720,1001,Contingency,NaN,1,0.0,LS,0,NaN,111.99,NaN,...,0,NaN,0,NaN,0,2023-11-28,10378 Brunner Farm-CGRS,1,2023,11
7722,1010,Field Supervision,NaN,0.6,0.0,WKS,0,NaN,645.24,NaN,...,0,NaN,1404,NaN,3571,2023-11-28,10378 Brunner Farm-CGRS,1,2023,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5010,25558,Furnish & Install VTC Tracking Pad,NaN,0,0.0,LS,NaN,NaN,762.8,-762.8,...,0,NaN,762.8,NaN,762.8,2022-01-03,10325-30 StoneGate-Lokal,2,2022,1
5009,25456,Remove & Dispose Of Existing Fence & Pump House,NaN,0,0.0,LS,NaN,NaN,0,NaN,...,0,NaN,0,NaN,0,2022-01-03,10325-30 StoneGate-Lokal,2,2022,1
5004,3043,Cut In Curb & Gutter,NaN,3972,0.0,LF,0,NaN,16688.81,1130.35,...,0,NaN,0,NaN,17819.17,2022-01-03,10325-30 StoneGate-Lokal,3,2022,1
5478,25699,Additional cut to stockpile and export,NaN,0,3240.0,LS,NaN,NaN,0,11071.08,...,1779,NaN,8354.87,NaN,8354.87,2022-01-03,10339-30 Victory South-Colorado Springs-Lokal,2,2022,1


In [26]:
len(last)

3154

In [9]:
most_recent = no_dup.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]
# df.drop_duplicates(subset=['IDName'], keep='last')


In [10]:
# subset of just earthwork
earth_work = most_recent[most_recent['Ph Int'] == '3']

In [19]:
crew_dict = {"Scarify":["Scarify"], "Strip Site":["Strip Site"], 
             "Soil Correction":["Soil Correction", "OX","OE", "Over Ex", "Excavate & Re-compact", "Over-Ex", "Excavate & Recompact"],
             "Ex Retainining Wall":["Excavate Site Retaining", "Excavate MSE", "Exc Retaining", "Exc Site Retn Walls"],
             "Export":["Export", "Exp"], "Import":["Import", "Imp"],
             "Ex Frost footing": ["Exc Fro", "Excavate Fro", "Footing Excavation"], 
             "Ex Trash Enc":["Excavate Trash", "Exc Trash"],
             "Cut to Fill":["Cut To Fill", "Utility Spoils - Site Fill", "Building Pad Fill w/ Spoils"],
             "Cut to Stockpile":["Cut to Stockpile", "Load & Stockpile"],
             "Handle spoils":["Handle Utility Spoils", "Handle Foundation Spoils", "Handle Spoils", "Handle & Stockpile"],
             "Double Handle":["Double Handle"], "Cut in C&G":["Cut In C", "Cut-in C", "Grade C&G"],
             "Ex Elevator":["Excavate Elevator"],
             "Grade Landscape":["grade Landscape", "grade Green", "Landscape"],
             "Grade Parkinglots and Roads":["Grade Boulevards","Grade Asphalt", "Grade Parking", "Grade Pkg", "Grade Main Roads"],
             "Grade SW":["Grade Sidewalks", "Grade Walks", "Exterior Slabs", "grade Brick Pavers", "Grade SW"],
             "Export":["Export"],
             "Grade BLDG": ["Grade Bldg", "Grade Building", "Grade Structural"],
             "Drain Tile":["Drain"],
             "Backfill Int":["Backfill Int", "Footing Backfill"],
             "Backfill Ext":["Backfill Ext"],
             "Backfill Elevator":["Backfill Elevator"],
             "Backfill Trash":["Backfill Trash"], "Backfill Frost": ["Backfill Frost"],
             "High Backfill": ["High Backfill"], "Grade Class X":["Grade Class"],
             "Excavate Interior":["Excavate Int", "Excavate Thickened", "Excavate Grade Beams", "Exc Int"],
             "Excavate Exterior":["Excavate Ext", "Exc Ext"], "Grade Pond":["Basin Grading","Grade Pond" , "Grade Detention Pond"],
             "Assist shoring":["Assist Shoring"],
             "Stairs":["Stairs"], "Place Strippings":["Place Stripping"],
             "Street Sweep":["Street Sweeping"]}

In [20]:
test = earth_work
test['crew'] = 'misc.'

C:\Users\willd\AppData\Local\Temp\ipykernel_11036\2156627274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crew'] = 'misc.'


In [21]:
for key, values in crew_dict.items():
    test.loc[test['Description'].str.lower().str.contains('|'.join([x.lower() for x in values])), "crew"] = key

In [22]:
reduced = test.loc[:, test.columns.notna()]
reduced = reduced.drop(['Notes','Lab','Eq', 'Reclass'], axis=1)

In [60]:
v = most_recent.Description.value_counts()
agg_df = most_recent[most_recent.Description.isin(v.index[v.gt(3)])]

In [24]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
reduced.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [62]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 917 entries, 0 to 3796
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Phase Code              917 non-null    object        
 1   Description             917 non-null    object        
 2   Notes                   0 non-null      object        
 3   Qty                     917 non-null    float64       
 4   HJ Qty Comp             917 non-null    float64       
 5   Unit                    917 non-null    object        
 6   % Comp                  913 non-null    object        
 7   nan                     0 non-null      float64       
 8   Budget Last Projection  916 non-null    object        
 9   New Budgets             58 non-null     object        
 10  Total Budget            917 non-null    float64       
 11  Actual Cost$            917 non-null    float64       
 12  Cmtd Cost$              44 non-null     object   